# Monitor de consulta

Describir las funciones del monitor


In [1]:
import os, json, yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import modulos.consultas as consultas

from ipywidgets import widgets

In [2]:
# Paths absolutos
__file__ = 'monitor_consulta.ipynb'
CURRENT = os.path.dirname(os.path.abspath(__file__))
ROOT = os.path.dirname(CURRENT)

# Archivo de configuración
with open(os.path.join(ROOT, 'config.yaml'), 'r') as f:
    config = yaml.safe_load(f)
f.close()

# Paths de datos
TIEMPO_PATH = os.path.join(ROOT, config['datos']['tiempo'])

# Interés en el tiempo
Carga de datos

In [3]:
with open(file=TIEMPO_PATH, mode='r') as file:
    contenido = json.load(file)
file.close()

## Porcentaje de criterios consultados (PCC)

$$PCC = \frac{\#\{criterios\,consultados\}}{\#\{criterios\,búsqueda\}}$$

In [4]:
# Criterios búsqueda
with open(file=os.path.join(ROOT, config['datos']['criterios']), mode='r') as f:
    criterios_busqueda = [row.strip() for row in f]        
f.close()

# Criterios consultados
consultados = consultas.criterios_existentes(file=TIEMPO_PATH)

**PCC**

In [5]:
pcc_tiempo = len(consultados) / len(criterios_busqueda)
print(f'Pct. Criterios Consultados: {pcc_tiempo*100:.2f}%')

Pct. Criterios Consultados: 85.11%


## Exploración tabular


In [6]:
def ver_tiempo(criterio):
    criterios = [contenido[k]['criterio'] for k in range(len(contenido))]
    idx_criterio = criterios.index(criterio)
    
    consulta = contenido[idx_criterio]['contenidos']
    
    return pd.DataFrame(consulta).head()

# consultas = list(contenido[0].keys())
# consultas.remove("criterio")

widgets.interact(ver_tiempo,
                 criterio=widgets.Dropdown(
                     options=[contenido[k]['criterio'] for k in range(len(contenido))],
                     description='Criterio',
                     disabled=False)
                )

interactive(children=(Dropdown(description='Criterio', options=('inai', 'semanas cotizadas', 'expediente medic…

<function __main__.ver_tiempo(criterio)>

## Visualización inicial

In [9]:
def plot_tiempo(criterio):
    # Filtro de datos
    criterios = [contenido[k]['criterio'] for k in range(len(contenido))]
    idx_criterio = criterios.index(criterio)
    consulta = contenido[idx_criterio]['contenidos']
    over_time = pd.DataFrame(consulta)
    over_time['fecha'] = pd.to_datetime(over_time['fecha'])
    
    # Visualización
    fig, ax = plt.subplots(figsize=(16,6))

    scatter = ax.scatter(x=over_time['fecha'], y=over_time['valor'], c=over_time['parcial'], cmap='Paired')
    ax.plot(over_time['fecha'], over_time['valor'], color='black', alpha=0.25)

    #Anotaciones y estilo
    ax.set_title(label=f'Interés a lo largo del tiempo - {criterio.title()}\nÚltimos {config["etl"]["anios"]} años', fontsize=14, loc='left')
    ax.legend(*scatter.legend_elements(), loc="upper left", title="Parcial")
    ax.grid(alpha=0.1)

    plt.show()

In [10]:
widgets.interact(plot_tiempo,
                 criterio=widgets.Dropdown(
                     options=[contenido[k]['criterio'] for k in range(len(contenido))],
                     description='Criterio',
                     disabled=False)
                )

interactive(children=(Dropdown(description='Criterio', options=('inai', 'semanas cotizadas', 'expediente medic…

<function __main__.plot_tiempo(criterio)>